In [30]:
# Para llevar un registro de los cambios
%pip install -q comet_ml
import comet_ml
comet_ml.init(project_name='Tarea 03')

In [31]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [32]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)
experiment.set_name("Regularización_L1")

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/6ef334a2f31047d2a007312ff210adb1
COMET INFO:   Others:
COMET INFO:     Name : Regularización_L1
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 10
COMET INFO:     epochs        : 20
COMET INFO:     learning_rate : 0.01
COMET INFO:     loss          : categorical_crossentropy
COMET INFO:     optimizer     : adam
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING:

In [33]:
# Carga el dataset de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [34]:
# Convierte y normaliza los datos de entrada
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [35]:
# to_categorical convierte las etiquetas (train, test) en matrices one-hot
num_classes = 10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [36]:
  # Diccionario con los parámetros que se registrarán en comet
  parameters = {
      'epochs' : 20,
      'batch_size' : 10,
      'learning_rate' : 0.010,
      'loss' : 'categorical_crossentropy',
      'optimizer' : 'adam'
  }
  experiment.log_parameters(parameters)

In [37]:
# Creación del modelo
model = Sequential()
# Es como una "primera capa" de tres capas
model.add(Input(shape = (28,28))) # Matrices de entrada de 28x28 (imáegenes)
model.add(Flatten()) # Convierte las matrices de 28x28 en matrices de 784x1
model.add(Dense(784, activation = 'sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(30, activation = 'relu', kernel_regularizer = regularizers.L1(0.01))) # Segunda capa
model.add(Dense(10, activation = 'softmax')) # Tercera capa

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_7 (Dense)             (None, 784)               615440    
                                                                 
 dropout_3 (Dropout)         (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 30)                23550     
                                                                 
 dense_9 (Dense)             (None, 10)                310       
                                                                 
Total params: 639300 (2.44 MB)
Trainable params: 639300 (2.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
# Checkpoint que guarda el mejor modelo (cuando la función de costo alcanza un mínimo)
checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')

In [39]:
# optimizer = RMSprop(learning_rate = 0.0005)
optimizer = 'adam'
model.compile(loss = parameters['loss'], optimizer = optimizer, metrics = ['accuracy'])
model.fit(x_train, y_trainc, batch_size = parameters['batch_size'], epochs = parameters['epochs'], verbose = 1, validation_data = (x_test, y_testc), callbacks = [checkpoint])

Epoch 1/20
5996/6000 [============================>.] - ETA: 0s - loss: 1.1472 - accuracy: 0.8495
Epoch 1: val_loss improved from inf to 0.69832, saving model to best_model.hdf5
6000/6000 [==============================] - 67s 11ms/step - loss: 1.1470 - accuracy: 0.8495 - val_loss: 0.6983 - val_accuracy: 0.9102
Epoch 2/20
   1/6000 [..............................] - ETA: 2:04 - loss: 0.7114 - accuracy: 0.9000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6000/6000 [==============================] - ETA: 0s - loss: 0.6936 - accuracy: 0.9075
Epoch 2: val_loss improved from 0.69832 to 0.58149, saving model to best_model.hdf5
6000/6000 [==============================] - 61s 10ms/step - loss: 0.6936 - accuracy: 0.9075 - val_loss: 0.5815 - val_accuracy: 0.9344
Epoch 3/20
5998/6000 [============================>.] - ETA: 0s - loss: 0.6188 - accuracy: 0.9218
Epoch 3: val_loss did not improve from 0.58149
6000/6000 [==============================] - 62s 10ms/step - loss: 0.6188 - accuracy: 0.9218 - val_loss: 0.6148 - val_accuracy: 0.9261
Epoch 4/20
6000/6000 [==============================] - ETA: 0s - loss: 0.5774 - accuracy: 0.9299
Epoch 4: val_loss improved from 0.58149 to 0.51078, saving model to best_model.hdf5
6000/6000 [==============================] - 60s 10ms/step - loss: 0.5774 - accuracy: 0.9299 - val_loss: 0.5108 - val_accuracy: 0.9476
Epoch 5/20
5995/6000 [============================>.] - ETA: 0s - loss: 0.5486 - accuracy: 0.9370

In [40]:
experiment.log_model('MNIST1', 'best_model.hdf5')
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/97660c807ff5458799e94714612a779d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]                  : (0.8494833111763, 0.9604833126068115)
COMET INFO:     batch_accuracy [12000]         : (0.10000000149011612, 1.0)
COMET INFO:     batch_loss [12000]             : (0.30545541644096375, 12.991772651672363)
COMET INFO:     epoch_duration [20]            : (59.40397269899995, 67.62160066800016)
COMET INFO:     loss [20]                      : (0.42144277691841125, 1.1470459699630737)
COMET INFO:     val_accuracy [20]              : (0.9101999998092651, 0.9679999947547913)
COMET INFO:    